Once you have organised all the files into your `raw_image_info.csv` file, we now need to generate the raw_label images to train our model. These will be calculated based off the raw_labels found in `raw_annotated_file_path`.

# Load dependencies

In [1]:
# %gui qt5
import torch
import torchio as tio
from torch.utils.data import DataLoader
import scipy.io
from scipy.ndimage import maximum_filter
import pandas as pd
import numpy as np
import napari
import h5py

# Generation of training data

We have volumes of micro-ct data that we are trying to label. However, the labels we currently have are 3d point locations, which isn't a format that our deep learning model can link spatially to our ct data. 

We want to convert these 3d point locations into a new "prediction volume". This is what we want our deep learning model to end up producing. It is also a format that our deep learning model can read, link spatially to our data and produce.

## Let's load in the info we need from the csv file

In [2]:
info = pd.read_csv('raw_image_info.csv')

info

,image_file_path,raw_annotated_file_path,check_status,needs_transform_fix,needs_cropping
0,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/ants/diurna...,good,False,False
1,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/fiddlercrab...,good,False,False
2,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/fiddlercrab...,needs regenerating with updated annotations fr...,False,False
3,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/fiddlercrab...,mostly done,False,True
4,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,//home/jake/projects/mctv_resfiles/fiddlercrab...,good,False,False
...,...,...,...,...,...
151,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,NaN,NaN,NaN,NaN
152,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,NaN,NaN,NaN,NaN
153,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,NaN,NaN,NaN,NaN
154,//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jak...,NaN,NaN,NaN,NaN


Let's view one of the label files to see where the data is

In [3]:
info.loc[0, 'raw_annotated_file_path']

'//home/jake/projects/mctv_resfiles/ants/diurnal_tarsata/tarsata.mat'

In [4]:
f = h5py.File(info.loc[0, 'raw_annotated_file_path'], mode='r')

f['save_dat'].visititems(lambda n, o:print(n, o))

ana <HDF5 group "/save_dat/ana" (1 members)>
ana/para <HDF5 group "/save_dat/ana/para" (11 members)>
ana/para/allow_splitting_rows_by_distance <HDF5 dataset "allow_splitting_rows_by_distance": shape (1, 1), type "<f8">
ana/para/distance_for_pointtext <HDF5 dataset "distance_for_pointtext": shape (1, 1), type "<f8">
ana/para/include_non_empty <HDF5 dataset "include_non_empty": shape (1, 1), type "<f8">
ana/para/main_marker_size <HDF5 dataset "main_marker_size": shape (1, 1), type "<f8">
ana/para/pt_th <HDF5 dataset "pt_th": shape (1, 1), type "<f8">
ana/para/row_th <HDF5 dataset "row_th": shape (1, 1), type "<f8">
ana/para/spline_stiff <HDF5 dataset "spline_stiff": shape (1, 1), type "<f8">
ana/para/spline_stiff_smooth <HDF5 dataset "spline_stiff_smooth": shape (1, 1), type "<f8">
ana/para/spline_stiff_soft_mult <HDF5 dataset "spline_stiff_soft_mult": shape (1, 1), type "<f8">
ana/para/spline_stiff_z_mult <HDF5 dataset "spline_stiff_z_mult": shape (1, 1), type "<f8">
ana/para/text_size 

Here's the dataset

In [5]:
pd.DataFrame(np.array(f['save_dat/data/marked']))

,0,1,2,3,4,5,6,7,8,9,...,5170,5171,5172,5173,5174,5175,5176,5177,5178,5179
0,210.378869,208.715220,203.161893,201.773182,216.326074,211.377609,207.015189,202.492341,200.279329,199.304796,...,745.026416,746.956193,749.140737,752.587520,755.880528,750.993639,750.950449,748.290773,750.992589,752.272633
1,610.125062,608.291494,617.583415,627.088487,597.115077,601.899662,603.774637,614.244095,624.459194,633.980652,...,443.376608,430.567993,420.227072,411.458105,403.947978,401.989690,397.802645,439.233097,430.750563,421.386247
2,618.403081,605.820510,597.841257,598.722290,622.821452,613.526117,597.957832,588.549029,587.718511,594.389523,...,417.117119,429.199554,438.165841,448.930036,460.621231,462.513364,472.234956,424.412174,436.990685,446.607978
3,11.000000,11.000000,11.000000,11.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,...,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,77.000000,77.000000,77.000000
4,52.000000,53.000000,54.000000,55.000000,51.000000,52.000000,53.000000,54.000000,55.000000,56.000000,...,42.000000,43.000000,44.000000,45.000000,46.000000,47.000000,48.000000,43.000000,44.000000,45.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
f['save_dat']['stack']['image_format'].value.tobytes()[::2].decode()

/tmp/ipykernel_24076/1891607671.py:1: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  f['save_dat']['stack']['image_format'].value.tobytes()[::2].decode()


'dicom'

Now let's do the same thing with scipy in the case where the matlab file is an old type

In [7]:
dir = info.loc[2, 'raw_annotated_file_path']

mat = scipy.io.loadmat(dir)

file_type = mat['save_dat'][0]['stack'][0]['image_format'][0][0][0]
classification = pd.DataFrame(mat['save_dat'][0]['data'][0][0][0][0]).iloc[:,5]
points = pd.DataFrame(mat['save_dat'][0]['data'][0][0][0][0]).iloc[:,[2, 1, 0]]

In [8]:
file_type

'nifti'

In [9]:
classification

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
13914    0.0
13915    0.0
13916    0.0
13917    0.0
13918    0.0
Name: 5, Length: 13919, dtype: float64

In [10]:
points

,2,1,0
0,492.100525,456.952484,814.715515
1,480.170630,447.510213,815.730097
2,503.717560,467.951080,814.715515
3,502.127380,481.467499,814.715515
4,488.699310,472.278168,817.098267
...,...,...,...
13914,222.343597,455.246357,263.753937
13915,234.833111,454.485577,257.645381
13916,284.391574,439.530726,803.305356
13917,397.746509,222.228610,389.085456


Here's the locations of the annotations in x, y and z space

In [11]:
pd.DataFrame(np.array(f['save_dat']['data']['marked'])).iloc[[2, 1, 0], :]

,0,1,2,3,4,5,6,7,8,9,...,5170,5171,5172,5173,5174,5175,5176,5177,5178,5179
2,618.403081,605.820510,597.841257,598.722290,622.821452,613.526117,597.957832,588.549029,587.718511,594.389523,...,417.117119,429.199554,438.165841,448.930036,460.621231,462.513364,472.234956,424.412174,436.990685,446.607978
1,610.125062,608.291494,617.583415,627.088487,597.115077,601.899662,603.774637,614.244095,624.459194,633.980652,...,443.376608,430.567993,420.227072,411.458105,403.947978,401.989690,397.802645,439.233097,430.750563,421.386247
0,210.378869,208.715220,203.161893,201.773182,216.326074,211.377609,207.015189,202.492341,200.279329,199.304796,...,745.026416,746.956193,749.140737,752.587520,755.880528,750.993639,750.950449,748.290773,750.992589,752.272633


Here's the category associated with the annotations

In [12]:
pd.DataFrame(np.array(f['save_dat']['data']['marked'])).iloc[5, :]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
5175    1.0
5176    1.0
5177    1.0
5178    1.0
5179    1.0
Name: 5, Length: 5180, dtype: float64

## Let's set up some helper functions to generate annotated prediction volumes for our analysis.

In [13]:
def _bool_3d_filter(array, indices, buffer=3):
    # TODO: do i need to subtract 1 to these because of differences in matlab and python indexing?
    for i in range(len(indices[0])):
        ind = (indices[0][i], indices[1][i], indices[2][i])
        x_min = max(ind[0] - buffer, 0)
        x_max = min(ind[0] + buffer, array.shape[1])
        y_min = max(ind[1] - buffer, 0)
        y_max = min(ind[1] + buffer, array.shape[2])
        z_min = max(ind[2] - buffer, 0)
        z_max = min(ind[2] + buffer, array.shape[3])
        
        array[0, x_min:x_max, y_min:y_max, z_min:z_max] = 1
    
    return array

def _load_point_data(dir, swap_xy):
    print('loading point data from .mat files...')
    # load classifications
    if h5py.is_hdf5(dir):
        f = h5py.File(dir, mode='r')
        classification = pd.DataFrame(np.array(f['save_dat']['data']['marked'])).iloc[5, :]
        file_type = f['save_dat']['stack']['image_format'].value.tobytes()[::2].decode()
        print(f'Detected {file_type} file type')
        if swap_xy:
            points = pd.DataFrame(np.array(f['save_dat']['data']['marked'])).iloc[[2, 0, 1], :].T
        else:
            points = pd.DataFrame(np.array(f['save_dat']['data']['marked'])).iloc[[2, 1, 0], :].T
    else:
        mat = scipy.io.loadmat(dir)
        classification = pd.DataFrame(mat['save_dat'][0]['data'][0][0][0][0]).iloc[:,5]
        file_type = mat['save_dat'][0]['stack'][0]['image_format'][0][0][0]
        print(f'Detected {file_type} file type')
        if swap_xy:
            points = pd.DataFrame(mat['save_dat'][0]['data'][0][0][0][0]).iloc[:,[2, 0, 1]]
        else:
            points = pd.DataFrame(mat['save_dat'][0]['data'][0][0][0][0]).iloc[:,[2, 1, 0]]
    
    points.columns = ['x', 'y', 'z']


    # convert back to numpy array and round to nearest voxel
    points = np.array(points)
    points = np.round(points).astype(int)

    # get corneas and rhabdom locations with x, y and z data
    cornea_indx = (classification == 0) | (classification == 2)
    rhabdom_indx = (classification == 1) | (classification == 3)
    cornea_locations = points[cornea_indx, :]
    rhabdom_locations = points[rhabdom_indx, :]

    return cornea_locations, rhabdom_locations

def _point_to_segmentation_vol(image, cornea_locations, rhabdom_locations, flipY):
    print('converting point data to segmentation volume...')
    # create empty matrix the size of original data
    print('creating an empty image')
    empty = image.copy().astype('bool_')
    empty[:, :, :] = 0
    
    print('copying empty images')
    corneas = empty.copy()
    rhabdoms = empty.copy()
    
#     print('assigning positions of corneas and rhabdoms')
#     corneas[
#         0,
#         cornea_locations[:, 2],
#         cornea_locations[:, 1],
#         cornea_locations[:, 0]
#     ] = 1
#     rhabdoms[
#         0,
#         rhabdom_locations[:, 2],
#         rhabdom_locations[:, 1],
#         rhabdom_locations[:, 0]
#     ] = 1
    
#     # now use a maximum filter to make points a slightly larger areak
#     # note, that maximum filter makes predictions a cube without rounded edges
#     # a gaussian filter may be more appropriate
#     print('running maximum filter')
#     corneas = maximum_filter(corneas, size=3)
#     rhabdoms = maximum_filter(rhabdoms, size=3)
    
    print('assigning positions of corneas and rhabdoms with buffer')
    corneas = _bool_3d_filter(
        corneas,
        (
            cornea_locations[:, 2],
            cornea_locations[:, 1],
            cornea_locations[:, 0]
        ),
        buffer=1
    )
    
    rhabdoms = _bool_3d_filter(
        rhabdoms,
        (
            rhabdom_locations[:, 2],
            rhabdom_locations[:, 1],
            rhabdom_locations[:, 0]
        ),
        buffer=1
    )
    
    # now merge both into a single prediction volume
    # 0 = nothing
    # 1 = cornea
    # 2 = rhabdom
    print('merging cornea and rhabdom images into single volume')

    prediction = empty.astype(np.int16)
    prediction[corneas] = 1
    prediction[rhabdoms] = 2

    if flipY:
        print('Flipping Y because file was annotated with dicom')
        prediction = np.flip(prediction, 2).copy()
    
    return prediction

def create_annotated_volumes(dir, image, swap_xy, flip_y):
    cornea_locations, rhabdom_locations = _load_point_data(dir, swap_xy)
    annotated_vol = _point_to_segmentation_vol(
        image,
        cornea_locations,
        rhabdom_locations,
        flip_y
    )
    print('done.')
    return annotated_vol
    

In [14]:
from pathlib import Path
import os

n_rows = info.shape[0]

out_label_dir = '//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/'

plot = False

for i in range(n_rows):
    img = info.loc[i, 'image_file_path']
    label = info.loc[i, 'raw_annotated_file_path']
    swap_xy = info.loc[i, 'swap_xy']
    flip_y = info.loc[i, 'flip_y']

    p = Path(img)
    filename = p.stem
    out_path = out_label_dir + filename + '.nii'
    
    if not os.path.isfile(out_path):
        print('starting conversion of ' + filename)
        transform = tio.ToCanonical()
        img = tio.ScalarImage(
            img
        )
        ann = tio.LabelMap(
            tensor=create_annotated_volumes(
                label,
                img.data.numpy(),
                swap_xy,
                flip_y
            ),
            affine=img.affine
        )
        
        
        if plot:
            viewer = napari.Viewer()
            viewer.dims.ndisplay = 3 # toggle 3 dimensional view
            viewer.add_image(img.data.numpy())
            viewer.add_image(ann.data.numpy())

        # now save the label/annotation
        print('saving to ' + out_path)
        ann.save(out_path)
    else:
        print(out_path + ' has already been created, so skipping')

Needs transform fix: False
//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/diurnal_tarsata.nii has already been created, so skipping
Needs transform fix: False
//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/dampieri_20151218.nii has already been created, so skipping
Needs transform fix: False
//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/dampieri_20200218_male_left_1676.nii has already been created, so skipping
Needs transform fix: False
//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/dampieri_male_16.nii has already been created, so skipping
Needs transform fix: False
//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/flammula_20180307.nii has already been created, so skipping
Needs transform fix: False
//mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/flammula_20190925_male_left.nii has already been created, so skipping
Needs transform fix: False
//m

/tmp/ipykernel_24076/2796752783.py:22: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  file_type = f['save_dat']['stack']['image_format'].value.tobytes()[::2].decode()


copying empty images
assigning positions of corneas and rhabdoms with buffer
merging cornea and rhabdom images into single volume
Flipping Y because file was annotated with dicom
done.
saving to //mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/P_gracilis_FEG190213_003a_head_HIGHESTpriority.nii
Needs transform fix: True
starting conversion of P_robisoni_FEG200205_105-head
loading point data from .mat files...
Detected auto file type
converting point data to segmentation volume...
creating an empty image


/tmp/ipykernel_24076/2796752783.py:22: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  file_type = f['save_dat']['stack']['image_format'].value.tobytes()[::2].decode()


copying empty images
assigning positions of corneas and rhabdoms with buffer
merging cornea and rhabdom images into single volume
Flipping Y because file was annotated with dicom
done.
saving to //mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/P_robisoni_FEG200205_105-head.nii
Needs transform fix: True
starting conversion of Paraphronima_FEG181024_03_head_sp3_1423158b
loading point data from .mat files...
Detected auto file type
converting point data to segmentation volume...
creating an empty image


...

/tmp/ipykernel_24076/2796752783.py:22: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  file_type = f['save_dat']['stack']['image_format'].value.tobytes()[::2].decode()


copying empty images
assigning positions of corneas and rhabdoms with buffer
merging cornea and rhabdom images into single volume
Flipping Y because file was annotated with dicom
done.
saving to //mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/streetsia_20190308.nii
Needs transform fix: True
starting conversion of streetsia_20190325
loading point data from .mat files...
Detected dicom file type
converting point data to segmentation volume...
creating an empty image


/tmp/ipykernel_24076/2796752783.py:22: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  file_type = f['save_dat']['stack']['image_format'].value.tobytes()[::2].decode()


copying empty images
assigning positions of corneas and rhabdoms with buffer
merging cornea and rhabdom images into single volume
Flipping Y because file was annotated with dicom
done.
saving to //mnt/d37c99c5-3b94-47b9-9965-c66fd9a16e23/jake/mctnet_data/raw_labels/streetsia_20190325.nii
Needs transform fix: nan
starting conversion of 20181221L_female_cropped
loading point data from .mat files...


TypeError: expected str, bytes or os.PathLike object, not float

Now, that we have our raw labels and images, let's move to step 03